# IMDB Film Puanlama ve Sıralama (IMDB Scoring and Sorting)

In [1]:
import pandas as pd 
import math
import scipy.stats as st 

In [3]:
df = pd.read_csv("datasets/movies_metadata.csv", low_memory=False)
#DtypeWarning kapamak için


In [104]:
df = df[["title", "vote_average", "vote_count"]]

In [105]:
df.head()

,title,vote_average,vote_count
0,Toy Story,7.7,5415.0
1,Jumanji,6.9,2413.0
2,Grumpier Old Men,6.5,92.0
3,Waiting to Exhale,6.1,34.0
4,Father of the Bride Part II,5.7,173.0


In [106]:
df.shape
#45466tane film var

(45466, 3)

In [107]:
# Örneğin vote_average a göre bir sıralama yapalım. 
df.sort_values("vote_average", ascending=False).head(20)
#Bu sıralama beklentimizin dışında oldu. Dolayısıyla sadece ortalamaya göre sıralama yapmak mantıklı değil. 
#çünkü vote_count değerleri birbirinden oldukça farklı görülmekte.

,title,vote_average,vote_count
21642,Ice Age Columbus: Who Were the First Americans?,10.0,1.0
15710,If God Is Willing and da Creek Don't Rise,10.0,1.0
22396,Meat the Truth,10.0,1.0
22395,Marvin Hamlisch: What He Did For Love,10.0,1.0
35343,Elaine Stritch: At Liberty,10.0,1.0
186,Reckless,10.0,1.0
45047,The Human Surge,10.0,1.0
22377,The Guide,10.0,1.0
22346,هیچ کجا هیچ کس,10.0,1.0
1634,Other Voices Other Rooms,10.0,1.0


In [108]:
#Öyleyse vote_count'a bir filtre koyabiliriz. Yani belirli bir sayının üzerinde olanları alabiliriz sadece. 
#Peki bu sayıyı nasıl belirleyeceğiz; 
df["vote_count"].describe([0.10, 0.25, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99]).T
#Filmlerin ortalama oy sayısı 109.897 civarında görünüyor. Yani biz de filtreyi bu ortalamanın üzerinde bir değerde tutabiliriz.
# Bu yüzden 400leri tercih ediyoruz.

count    45460.000000
mean       109.897338
std        491.310374
min          0.000000
10%          1.000000
25%          3.000000
50%         10.000000
70%         25.000000
80%         50.000000
90%        160.000000
95%        434.000000
99%       2183.820000
max      14075.000000
Name: vote_count, dtype: float64

In [109]:
# Şimdi oy sayısı 400den büyük olanları seçip sıralayalım;
df[df["vote_count"] > 400].sort_values("vote_average", ascending=False).head(20)
# Sadece vote_count a göre filtre koymakta tam anlamıyla doğru mu bilemiyoruz. Bu yüzden daha kombine bir sistem oluşturmalıyız. 


,title,vote_average,vote_count
10309,Dilwale Dulhania Le Jayenge,9.1,661.0
40251,Your Name.,8.5,1030.0
834,The Godfather,8.5,6024.0
314,The Shawshank Redemption,8.5,8358.0
1152,One Flew Over the Cuckoo's Nest,8.3,3001.0
1176,Psycho,8.3,2405.0
1178,The Godfather: Part II,8.3,3418.0
292,Pulp Fiction,8.3,8670.0
1184,Once Upon a Time in America,8.3,1104.0
5481,Spirited Away,8.3,3968.0


In [110]:
# Bir minmaxscaler metodu getirip, vote_count değişkenini standartlaştıracağız. 1 ile 10 arasında değerler atamasını bekliyoruz.
from sklearn.preprocessing import MinMaxScaler

In [111]:
df["vote_count_score"]=MinMaxScaler(feature_range=(1, 10)). \
    fit(df[["vote_count"]]). \
    transform(df[["vote_count"]])

In [112]:
df

,title,vote_average,vote_count,vote_count_score
0,Toy Story,7.7,5415.0,4.462522
1,Jumanji,6.9,2413.0,2.542948
2,Grumpier Old Men,6.5,92.0,1.058828
3,Waiting to Exhale,6.1,34.0,1.021741
4,Father of the Bride Part II,5.7,173.0,1.110622
...,...,...,...,...
45461,Subdue,4.0,1.0,1.000639
45462,Century of Birthing,9.0,3.0,1.001918
45463,Betrayal,3.8,6.0,1.003837
45464,Satan Triumphant,0.0,0.0,1.000000


In [113]:
# vote_average değeri ve vote_count_score değerini çarparak average_count_score değeri oluşturuyoruz. 
df["average_count_score"]= df["vote_count_score"] * df["vote_average"]

In [114]:
#ş imdi ise average_count_score a göre sıralayalım
df.sort_values("average_count_score", ascending=False).head(20)
# ortalama olarak doğru bir sıralama olmuş gibi görünüyor. Tabii bu da yeterli değil. 

,title,vote_average,vote_count,vote_count_score,average_count_score
15480,Inception,8.1,14075.0,10.000000,81.000000
12481,The Dark Knight,8.3,12269.0,8.845187,73.415048
22879,Interstellar,8.1,11187.0,8.153321,66.041904
17818,The Avengers,7.4,12000.0,8.673179,64.181528
14551,Avatar,7.2,12114.0,8.746075,62.971737
26564,Deadpool,7.4,11444.0,8.317655,61.550650
2843,Fight Club,8.3,9678.0,7.188419,59.663879
20051,Django Unchained,7.8,10297.0,7.584227,59.156973
23753,Guardians of the Galaxy,7.9,10014.0,7.403268,58.485819
292,Pulp Fiction,8.3,8670.0,6.543872,54.314139


# IMDB Ağırlıklı Derecelendirme (IMDB Weighted Rating)

* weighted_rating = (v/(v+M) * r) + (M/(v+M) * C)
* r = vote average Filmin puanı
* v = vote count Oy sayısı
* M = minimum votes required to be listed in the Top 250 Sıralamaya girmek için gerekli olan min oy değeri
* C = the mean vote across the whole report (currently 7.0) Kitlenin genel ortalaması 


In [115]:
# Formülün ilk kısmında; 
# bir film gerekenden çok oy aldıysa bu filmin puanına uygulanacak olan düzeltmenin şiddeti az olacaktır. 
# bir film gereknden az oy aldıysa bu film listeye giremeyecektir çünkü çok fazla düzeltmeye maruz kalacaktır. 
#Formulün ikinci partında;
#oy sayısı yükseldikçe dezavantaj gibi oluyor, Kitlenin genel ortalaması kritiktir. 

In [116]:
M = 2500
C = df['vote_average'].mean()

def weighted_rating(r, v, M, C):
    return (v / (v + M) * r) + (M / (v + M) * C)

In [117]:
df.sort_values("average_count_score", ascending=False).head(20)
# Deadpool filmi beklentinin üzerinde bir sırada gibi görünüyor. Bunun için hesaplama yapalım. 

,title,vote_average,vote_count,vote_count_score,average_count_score
15480,Inception,8.1,14075.0,10.000000,81.000000
12481,The Dark Knight,8.3,12269.0,8.845187,73.415048
22879,Interstellar,8.1,11187.0,8.153321,66.041904
17818,The Avengers,7.4,12000.0,8.673179,64.181528
14551,Avatar,7.2,12114.0,8.746075,62.971737
26564,Deadpool,7.4,11444.0,8.317655,61.550650
2843,Fight Club,8.3,9678.0,7.188419,59.663879
20051,Django Unchained,7.8,10297.0,7.584227,59.156973
23753,Guardians of the Galaxy,7.9,10014.0,7.403268,58.485819
292,Pulp Fiction,8.3,8670.0,6.543872,54.314139


In [118]:
# Deadpool'un r değeri; 7.4, v değeri;11444.0, M ve C değerlerini yukarda formülize ettik. 
weighted_rating(7.4, 11444.0, M, C)


7.080544896574546

In [119]:
# Şimdi ise Inception için bakalım;
weighted_rating(8.1, 14075.0, M, C)

7.725672279809078

In [120]:
# Son olarak The Shawshank Redemption filmini değerlendirelim;
weighted_rating (8.5, 8358.0, M, C)

7.83648167598411

In [121]:
# Bu üç değer bizimkinden farklı dağıldı, bunu bütün veriye uygulayalım. 
df["weighted_rating"] = weighted_rating(df["vote_average"],
                                        df["vote_count"], M, C)

df.sort_values("weighted_rating", ascending=False).head(10)

,title,vote_average,vote_count,vote_count_score,average_count_score,weighted_rating
12481,The Dark Knight,8.3,12269.0,8.845187,73.415048,7.846044
314,The Shawshank Redemption,8.5,8358.0,6.344369,53.927140,7.836482
2843,Fight Club,8.3,9678.0,7.188419,59.663879,7.749460
15480,Inception,8.1,14075.0,10.000000,81.000000,7.725672
292,Pulp Fiction,8.3,8670.0,6.543872,54.314139,7.699778
834,The Godfather,8.5,6024.0,4.851936,41.241456,7.654800
22879,Interstellar,8.1,11187.0,8.153321,66.041904,7.646688
351,Forrest Gump,8.2,8147.0,6.209449,50.917485,7.593775
7000,The Lord of the Rings: The Return of the King,8.1,8226.0,6.259964,50.705712,7.521547
4863,The Lord of the Rings: The Fellowship of the Ring,8.0,8892.0,6.685826,53.486607,7.477310


In [122]:
# Şimdi bizimkini getirelim; 
df.sort_values("average_count_score", ascending=False).head(10)

,title,vote_average,vote_count,vote_count_score,average_count_score,weighted_rating
15480,Inception,8.1,14075.0,10.000000,81.000000,7.725672
12481,The Dark Knight,8.3,12269.0,8.845187,73.415048,7.846044
22879,Interstellar,8.1,11187.0,8.153321,66.041904,7.646688
17818,The Avengers,7.4,12000.0,8.673179,64.181528,7.092794
14551,Avatar,7.2,12114.0,8.746075,62.971737,6.929405
26564,Deadpool,7.4,11444.0,8.317655,61.550650,7.080545
2843,Fight Club,8.3,9678.0,7.188419,59.663879,7.749460
20051,Django Unchained,7.8,10297.0,7.584227,59.156973,7.373769
23753,Guardians of the Galaxy,7.9,10014.0,7.403268,58.485819,7.444152
292,Pulp Fiction,8.3,8670.0,6.543872,54.314139,7.699778


# Bayesian Average Rating Score (Bar Score)

In [123]:
# Yukardaki tüm yöntemler 2015 yıllarında gözlemlenmiştir. 
# IMDB'nin sitesinde sıralama değişmiştir. IMDB yeni bir sıralama yöntemi geliştirmiştir. 
# Bu sıralamaya en yakın çıktıyo elde etmek için bayesian average rating deneyeceğiz. 




In [124]:
def bayesian_average_rating(n, confidence=0.95):
    if sum(n) == 0:
        return 0
    K = len(n)
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    N = sum(n)
    first_part = 0.0
    second_part = 0.0
    for k, n_k in enumerate(n):
        first_part += (k + 1) * (n[k] + 1) / (N + K)
        second_part += (k + 1) * (k + 1) * (n[k] + 1) / (N + K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score
bayesian_average_rating([34733, 4355, 4704, 6561, 13515, 26183, 87368, 273082, 600260, 1295351])#esaretin bedeli için tüm yıldız sayılarını yazalım. 
# IMDB'deki güncel puan 9.3. 

9.14538444560111

In [125]:
#Şimdi yukardaki işlemi baba filmi için uygulayalım;
bayesian_average_rating([37128, 5879, 6268, 8419, 16603, 30016, 78538, 199430, 402518, 837905]) #kaç 1 yıldız, kaç 2 yıldız.... kaç 10 yıldız
#Güncel puan 9.2 olarak görülmekte. 
#user Quality i biliyor olsaydık IMDB puanına daha çok yaklaşırdık. 

8.940007324860396

In [126]:
#Şimdi bütün bunları filmlerin hepsine uygulayalım; 
df= pd.read_csv("/Users/elifbagci/Desktop/Miuul:DATA SCİENCE/Elif Miuul notlar/Measurement Problems/Uygulama- IMDB /dataset/imdb_ratings.csv")
df=df.iloc[0:, 1:]

In [127]:
df.head()
# Kullandığımız bu yeni verisetinden hangi yıldızın kaç tane aldığını görebiliriz. 


,id,movieName,rating,ten,nine,eight,seven,six,five,four,three,two,one
0,111161,1. The Shawshank Redemption (1994),9.2,1295382,600284,273091,87368,26184,13515,6561,4704,4355,34733
1,68646,2. The Godfather (1972),9.1,837932,402527,199440,78541,30016,16603,8419,6268,5879,37128
2,71562,3. The Godfather: Part II (1974),9.0,486356,324905,175507,70847,26349,12657,6210,4347,3892,20469
3,468569,4. The Dark Knight (2008),9.0,1034863,649123,354610,137748,49483,23237,11429,8082,7173,30345
4,50083,5. 12 Angry Men (1957),8.9,246765,225437,133998,48341,15773,6278,2866,1723,1478,8318


In [128]:
df["bar_score"] = df.apply(lambda x: bayesian_average_rating(x[["one", "two", "three", "four", "five",
                                                                "six", "seven", "eight", "nine", "ten"]]), axis=1)
df.sort_values("bar_score", ascending=False).head(20)

,id,movieName,rating,ten,nine,eight,seven,six,five,four,three,two,one,bar_score
0,111161,1. The Shawshank Redemption (1994),9.2,1295382,600284,273091,87368,26184,13515,6561,4704,4355,34733,9.145389
1,68646,2. The Godfather (1972),9.1,837932,402527,199440,78541,30016,16603,8419,6268,5879,37128,8.940016
3,468569,4. The Dark Knight (2008),9.0,1034863,649123,354610,137748,49483,23237,11429,8082,7173,30345,8.895962
2,71562,3. The Godfather: Part II (1974),9.0,486356,324905,175507,70847,26349,12657,6210,4347,3892,20469,8.812499
4,50083,5. 12 Angry Men (1957),8.9,246765,225437,133998,48341,15773,6278,2866,1723,1478,8318,8.767934
6,167260,7. The Lord of the Rings: The Return of ...,8.9,703093,433087,270113,117411,44760,21818,10873,7987,6554,28990,8.752038
5,108052,6. Schindler's List (1993),8.9,453906,383584,220586,82367,27219,12922,6234,4572,4289,19328,8.743609
11,109830,12. Forrest Gump (1994),8.8,622104,553654,373644,151284,51140,22720,11692,7647,5941,12110,8.699152
12,1375666,13. Inception (2010),8.7,724798,627987,408686,174229,60668,26910,13436,8703,6932,17621,8.693148
10,137523,11. Fight Club (1999),8.8,637087,572654,371752,152295,53059,24755,12648,8606,6948,17435,8.674475


In [129]:
#Yukarda görüldüğü üzere IMDB ile çok yakın bir sırlama yaptık. 👍🏻